In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# Get support articles
We could copy and paste, but we didn't learn to code to copy and paste

In [3]:
SUPPORT_HOME_URL = 'https://support.mycase.com/en/'
support_home_page = requests.get(SUPPORT_HOME_URL)
soup = BeautifulSoup(support_home_page.text, 'html.parser')


In [4]:
parent_links = []
for parent_link in soup.find_all('a', class_='collection-link'):
    parent_links.append(parent_link.get('href'))

In [5]:
def extract_child_links(parent_link):
    parent_link_page = requests.get(parent_link)
    soup = BeautifulSoup(parent_link_page.text, 'html.parser')
    result = []
    for link in soup.find_all('a', class_='duration-250'):
        result.append(link.get('href'))
    return result

In [6]:
all_links = []
link_dict = {}
for link in parent_links:
    child_links = extract_child_links(link)
    link_dict[link] = child_links
    all_links += child_links
    time.sleep(3)

# Create Embeddings and Document
Use for Document Q/A

In [1]:
from langchain.document_loaders import UnstructuredURLLoader
import nltk

In [9]:
loader = UnstructuredURLLoader(urls=all_links)

In [11]:
data = loader.load()

In [19]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes import VectorstoreIndexCreator
import os
os.environ["OPENAI_API_KEY"] = '...'

In [15]:
HUGGING_FACE_API_KEY = "..."
hf_embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/Users/michael.branconier/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████████████████████| 1.18k/1.18k [00:00<00:00, 408kB/s]
Downloading: 100%|█████████████████████████████| 190/190 [00:00<00:00, 47.1kB/s]
Downloading: 100%|█████████████████████████| 10.6k/10.6k [00:00<00:00, 1.99MB/s]
Downloading: 100%|█████████████████████████████| 116/116 [00:00<00:00, 41.4kB/s]
Downloading: 100%|██████████████████████████| 39.3k/39.3k [00:00<00:00, 601kB/s]
Downloading: 100%|█████████████████████████| 90.9M/90.9M [00:03<00:00, 24.5MB/s]
Downloading: 100%|███████████████████████████| 53.0/53.0 [00:00<00:00, 11.0kB/s]
Downloading: 100%|█████████████████████████████| 112/112 [00:00<00:00, 20.1kB/s]
Downloading: 100%|███████████████████████████| 466k/466k [00:00<00:00, 2.17MB/s]
D

In [20]:
index = VectorstoreIndexCreator(
    vectorstore_cls=Chroma, 
    embedding=hf_embeddings,
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
).from_loaders([loader]) 

Created a chunk of size 1206, which is longer than the specified 1000
Created a chunk of size 1006, which is longer than the specified 1000
Created a chunk of size 1071, which is longer than the specified 1000
Created a chunk of size 1092, which is longer than the specified 1000
Created a chunk of size 1045, which is longer than the specified 1000
Created a chunk of size 1015, which is longer than the specified 1000
Created a chunk of size 1270, which is longer than the specified 1000
Created a chunk of size 1184, which is longer than the specified 1000
Created a chunk of size 1122, which is longer than the specified 1000
Created a chunk of size 1093, which is longer than the specified 1000
Created a chunk of size 1025, which is longer than the specified 1000
Using embedded DuckDB without persistence: data will be transient


In [24]:
query = "How do I edit an invoice?"
index.query_with_sources(query)

{'question': 'How do I edit an invoice?',
 'answer': ' To edit an invoice, find the invoice you want to edit and open its details page. Then, in the actions toolbar, click the Edit button. You can also edit invoice sharing, payment history, expense entries, unpaid invoice balance forward, adjustments, terms and conditions/notes.\nSource: https://support.mycase.com/en/articles/6638008-how-do-i-edit-an-existing-invoice-w-video, https://support.mycase.com/en/articles/6081993-creating-an-invoice-w-video, https://support.mycase.com/en/articles/6403155-marking-invoices-as-sent, https://support.mycase.com/en/articles/6081934-billing-invoicing-guide',
 'sources': ''}

In [25]:
query = "How do I Manually Import Bank Feed Transactions?"
index.query_with_sources(query)

{'question': 'How do I Manually Import Bank Feed Transactions?',
 'answer': ' To manually import bank feed transactions, you need to upload the file to the MyCase application, process the transactions, map the columns, and view the result.\n',
 'sources': 'https://support.mycase.com/en/articles/6577421-how-to-manually-import-bank-feed-transactions'}

In [26]:
query = "How do I perform banknig reconciliations?"
index.query_with_sources(query)

{'question': 'How do I perform banknig reconciliations?',
 'answer': ' To perform bank reconciliations, begin by matching and categorizing all transactions within the statement period. Then, navigate to the Accounting tab and select Bank Reconciliation. Select the bank account to reconcile, review the Beginning Balance, choose the Ending Date, and select Start Reconciling. Compare the bank statement to the transactions entered in MyCase, and check the box associated with each transaction to reconcile it. If the difference between the statement and MyCase balance is $0.00, select the option to Save and Finish.\n',
 'sources': 'https://support.mycase.com/en/articles/6263648-performing-bank-reconciliations-in-mycase-accounting'}

In [27]:
query = "How do I install the outlook add in?"
index.query_with_sources(query)

{'question': 'How do I install the outlook add in?',
 'answer': ' To install the Outlook add-in, click the 3 dots in line with the recipient details, select Get Add-ins, search “MyCase” in the Add-in store, click Add to install, open any email in your inbox, click the 3 dots in line with the recipient details, select MyCase Add-in, and select Link Email to MyCase.\n',
 'sources': 'https://support.mycase.com/en/articles/7060992-mycase-outlook-add-in-usage-guide\nhttps://support.mycase.com/en/articles/6449956-uninstalling-the-old-mycase-outlook-plugin\nhttps://support.mycase.com/en/articles/7048956-install-document-automation-add-in-manually-with-xml-manifest-file'}

In [28]:
query = "How do you create a new time entry?"
index.query_with_sources(query)

{'question': 'How do you create a new time entry?',
 'answer': ' To create a new time entry, click the Add Time Entry on the dashboard or Add Item Menu. You will be taken to the Add Time Entry window where you can specify the case/matter, user, and activity associated with the time entry.\n',
 'sources': 'https://support.mycase.com/en/articles/6105927-time-tracking-w-video, https://support.mycase.com/en/articles/6110080-billing-activities, https://support.mycase.com/en/articles/6153250-creating-a-new-calendar-event'}